In [1]:
import transformers, datasets, torch
import torch
import numpy as np
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollator, Trainer, TrainingArguments
from datasets import Dataset, load_metric, load_dataset, Features, Value
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, recall_score

2023-03-20 10:52:51.125282: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Dataset info

In [2]:
dss = load_dataset('EvaKlimentova/knots_AF')
print(dss)

Using custom data configuration EvaKlimentova--knots_AF-293560de9ceccb3f
Found cached dataset parquet (/home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--knots_AF-293560de9ceccb3f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
        num_rows: 39412
    })
    train: Dataset({
        features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
        num_rows: 157644
    })
})


In [4]:
print(dss['train'].filter(lambda x: x['label'] == 0))
print(dss['test'].filter(lambda x: x['label'] == 0))

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--knots_AF-293560de9ceccb3f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-4c60d0b73ff9532e.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--knots_AF-293560de9ceccb3f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-07d32d2becbfd561.arrow


Dataset({
    features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
    num_rows: 78810
})
Dataset({
    features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
    num_rows: 19718
})


In [5]:
print(dss['train'].filter(lambda x: x['label'] == 1))
print(dss['test'].filter(lambda x: x['label'] == 1))

  0%|          | 0/158 [00:00<?, ?ba/s]

Dataset({
    features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
    num_rows: 78834
})


  0%|          | 0/40 [00:00<?, ?ba/s]

Dataset({
    features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
    num_rows: 19694
})


In [7]:
print(dss['train'].filter(lambda x: x['FamilyName'] == 'PGluconate dehydrogenase'))
print(dss['test'].filter(lambda x: x['FamilyName'] == 'PGluconate dehydrogenase'))

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--knots_AF-293560de9ceccb3f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-7b6e535b4791517e.arrow


Dataset({
    features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
    num_rows: 0
})


  0%|          | 0/40 [00:00<?, ?ba/s]

Dataset({
    features: ['ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'uniprotSequence', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'label', 'FamilyName'],
    num_rows: 0
})


# Model evaluation

In [3]:
def tokenize_function(s):
  seq_split = " ".join(s['uniprotSequence'])
  return tokenizerM1(seq_split)

In [2]:
tokenizerM1 = AutoTokenizer.from_pretrained("roa7n/knots_protbertBFD_alphafold")
modelM1 = AutoModelForSequenceClassification.from_pretrained("roa7n/knots_protbertBFD_alphafold")

In [6]:
modelM1.num_parameters()

419933186

In [17]:
model_parameters = filter(lambda p: p.requires_grad, modelM1.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

419933186

In [5]:
family_mapping = {
'IPR029026': 'SPOUT',
'IPR004837': 'Sodium/calcium exchanger',
'IPR036398': 'Carbonic anhydrase',
'IPR036901': 'ATCase/OTCase',
'IPR006131': 'ATCase/OTCase',
'IPR032695': 'Integrin alpha',
'IPR013649': 'Integrin alpha',
'IPR004813': 'OPT Oligopeptide transporter',
'IPR013694': 'ITIH',
'IPR018723': 'DUF2254 membrane',
'IPR002035': 'ITIH',
'IPR022628': 'AdoMet synthase',
'IPR002133': 'AdoMet synthase',
'IPR022636': 'AdoMet synthase',
'IPR003929': 'Calcium-activated potassium channel BK, alpha subunit',
'IPR001578': 'UCH',
'IPR036959': 'UCH',
'IPR006827': 'Lantibiotic dehydratase',
'IPR005636': 'TDD',
'IPR038459': 'SPOUT',
'IPR007209': 'TDD',
'IPR022968': 'TDD',
'IPR007177': 'TDD',
'IPR010793': 'Ribosomal protein L37/S30',
'IPR025349': 'DUF4253',
'IPR027790': 'AdoMet synthase',
'IPR042544': 'AdoMet synthase',
'IPR002795': 'AdoMet synthase',
'IPR018883': 'Carbonic anhydrase'
}

In [6]:
def map_family(row):
    ipro = row['InterPro']
    if pd.isna(ipro):
        row['FamilyName'] = ''
        return row
    ipro = ipro.split(';')
    families = set()
    for fam in ipro:
        if fam in family_mapping.keys():
            families.add(family_mapping[fam])
    row['FamilyName'] = ','.join(list(families))
    return row

In [7]:
test_dss = dss['test'].map(map_family)
test_df = pd.DataFrame(test_dss)

family_groups = test_df.groupby("FamilyName")

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/EvaKlimentova___parquet/EvaKlimentova--knots_AF-293560de9ceccb3f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-b4031a95253fbe4c.arrow


In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    specificity=recall_score(labels, preds, pos_label=0)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall (TPR)': recall,
        'specificity (TNR)': specificity
    }

In [ ]:
table = "Name, Size, Negatives, Accuracy, TPR, TNR \n"
for i, df in family_groups:
    # print(family)
    dataset = Dataset.from_pandas(df)
    dataset = dataset.map(tokenize_function, remove_columns=['uniprotSequence', 'ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'FamilyName'], num_proc=4)
    dataset.set_format("pt")

    training_args = TrainingArguments('outputs', fp16=True, per_device_eval_batch_size=1, report_to='none')  
    trainer = Trainer(
        modelM1,
        training_args,
        train_dataset=dataset,
        eval_dataset=dataset,
        tokenizer=tokenizerM1,
        compute_metrics=compute_metrics
    )

    metrics = trainer.evaluate()
    table = table + f"{i}, {df.shape[0]}, {df[df['label'] == 0].shape[0]}, {round(metrics['eval_accuracy'], 4)}, {round(metrics['eval_recall (TPR)'], 4)}, {round(metrics['eval_specificity (TNR)'], 4)} \n"

In [16]:
print(table)

Name, Size, Negatives, Accuracy, TPR, TNR 
, 1, 0, 1.0, 1.0, 0.0 
ATCase/OTCase, 3799, 3352, 0.9995, 0.9978, 0.9997 
AdoMet synthase, 1794, 240, 0.99, 0.9929, 0.9708 
Calcium-activated potassium channel BK, alpha subunit, 127, 87, 0.8898, 0.975, 0.8506 
Carbonic anhydrase, 1531, 539, 0.9589, 0.9738, 0.9314 
DUF2254 membrane, 593, 376, 0.9983, 0.9954, 1.0 
DUF4253, 123, 53, 0.8618, 0.9, 0.8113 
ITIH, 14263, 12555, 0.9872, 0.942, 0.9934 
ITIH,Integrin alpha, 84, 84, 1.0, 0.0, 1.0 
Integrin alpha, 332, 224, 0.8313, 0.6667, 0.9107 
Lantibiotic dehydratase, 392, 286, 0.9643, 0.9528, 0.9685 
OPT Oligopeptide transporter, 2510, 456, 0.9865, 0.9951, 0.9474 
Ribosomal protein L37/S30, 147, 41, 0.8571, 1.0, 0.4878 
SPOUT, 7371, 550, 0.9889, 0.9953, 0.9091 
Sodium/calcium exchanger, 5256, 726, 0.9905, 0.9965, 0.9532 
TDD, 612, 24, 0.9902, 0.9966, 0.8333 
UCH, 477, 125, 0.9057, 0.9602, 0.752 



In [18]:
test_dss = test_dss.map(tokenize_function, remove_columns=['uniprotSequence', 'ID', 'latestVersion', 'globalMetricValue', 'uniprotStart', 'uniprotEnd', 'Length', 'Domain_architecture', 'InterPro', 'Max_Topology', 'Max Freq', 'Knot Core', 'FamilyName'], num_proc=4)
test_dss.set_format("pt")


training_args = TrainingArguments('outputs', fp16=True, per_device_eval_batch_size=1, report_to='none')  
trainer = Trainer(
    modelM1,
    training_args,
    train_dataset=test_dss,
    eval_dataset=test_dss,
    tokenizer=tokenizerM1,
    compute_metrics=compute_metrics
)

trainer.evaluate()

#0:   0%|          | 0/9853 [00:00<?, ?ex/s]

#1:   0%|          | 0/9853 [00:00<?, ?ex/s]

#2:   0%|          | 0/9853 [00:00<?, ?ex/s]

#3:   0%|          | 0/9853 [00:00<?, ?ex/s]

PyTorch: setting up devices
Using cuda_amp half precision backend
***** Running Evaluation *****
  Num examples = 39412
  Batch size = 1


{'eval_loss': 0.0637286975979805,
 'eval_accuracy': 0.9846493453770425,
 'eval_f1': 0.9846722910491247,
 'eval_precision': 0.9826060575415887,
 'eval_recall (TPR)': 0.9867472326596933,
 'eval_specificity (TNR)': 0.9825540115630389,
 'eval_runtime': 872.5216,
 'eval_samples_per_second': 45.17,
 'eval_steps_per_second': 45.17}

------------------------------------------

In [9]:
from torch import nn
from huggingface_hub import hf_hub_download

In [11]:
class CNN(nn.Module):
    def __init__(self, device):
        super(CNN, self).__init__()
        
        self.device = device

        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=7, padding=0)
        self.batchnorm = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()
        self.flat = nn.Flatten()
        self.lin1 = nn.Linear(32576, 512)
        self.lin2 = nn.Linear(512, 1)
        self.sigmoid = nn.Sigmoid()
        
        self.loss = nn.functional.binary_cross_entropy
 

    def forward(self, X):
        X = self.conv1(X)
        X = self.batchnorm(X)
        X = self.relu(X)
        X = self.flat(X)
        X = self.lin1(X)
        X = self.lin2(X)
        X = self.sigmoid(X)
        return X
    
    def train_model(self, dataset, epochs):  
        model.train()
        optimizer = torch.optim.Adam(self.parameters())
        
        for epoch in range(epochs):
            with tqdm(dataset, unit="batch") as tepoch:
                for inputs, targets in tepoch:
                    
                    inputs, targets = inputs.to(self.device), targets.to(self.device)
                    tepoch.set_description(f"Epoch {epoch + 1}")
                    
                    # clear the gradients
                    optimizer.zero_grad()
                    # compute the model output
                    yhat = self(inputs.float())
                    # calculate accuracy
                    correct = (torch.round(yhat) == targets).sum().item()
                    accuracy = correct / len(inputs)
                    # calculate loss
                    loss = self.loss(yhat, targets.float())
                    # credit assignment
                    loss.backward()
                    # update model weights
                    optimizer.step()
                    
                    tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
                
    def test(self, dataloader):
        model.eval()
        predictions, actuals = list(), list()
        
        with torch.no_grad():
            for inputs, targets in dataloader:
                inputs = inputs.to(self.device)
                
                # evaluate the model on the test set
                yhat = self(inputs.float())
                yhat = yhat.cpu().detach().numpy()
                actual = targets.numpy()
                # reshape for stacking
                actual = actual.reshape((len(actual), 1))
                yhat = yhat.reshape((len(yhat), 1))
                # store
                predictions.append(yhat)
                actuals.append(actual)
        predictions, actuals = np.vstack(predictions), np.vstack(actuals)
        print("Predictions: ", predictions[:10])
        print("Real labels: ", actuals[:10])
        # calculate accuracy
        pred_label = np.round(predictions)
        acc = metrics.accuracy_score(actuals, pred_label)
        f1 = metrics.f1_score(actuals, pred_label, average='binary', zero_division=0)
        precision, recall, thresholds = metrics.precision_recall_curve(actuals, predictions)
        auprc = metrics.auc(recall, precision)
        print(f"Test metrics: \n Accuracy: {float(acc):>6f}, F1 score: {float(f1):>6f}, AUPRC: {float(auprc):>6f}\n")
        return acc, f1, auprc


In [12]:
def test_model(model, dataloader):
    predictions, actuals = list(), list()
        
    with torch.no_grad():
            for inputs, targets in dataloader:
                inputs = inputs.to(device)
                
                # evaluate the model on the test set
                yhat = model(inputs.float())
                yhat = yhat.cpu().detach().numpy()
                actual = targets.numpy()
                # reshape for stacking
                actual = actual.reshape((len(actual), 1))
                yhat = yhat.reshape((len(yhat), 1))
                # store
                predictions.append(yhat)
                actuals.append(actual)
    predictions, actuals = np.vstack(predictions), np.vstack(actuals)
    return predictions

In [22]:
path = hf_hub_download(repo_id="EvaKlimentova/knots_M2_embeddings_alphafold", filename="M2_embedding_CNN.pth")

In [23]:
model = torch.load(path)
model.eval()

CNN(
  (conv1): Conv1d(1, 32, kernel_size=(7,), stride=(1,))
  (batchnorm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (flat): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=32576, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [26]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

16680257


------------------------------------------

In [19]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [20]:
path = hf_hub_download(repo_id="roa7n/knots_simple_CNN", filename="cnn_10epochs_94.h5")
model = tf.keras.models.load_model(path)

2023-03-20 12:54:51.631244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-20 12:54:51.644882: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 72408 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:21:00.0, compute capability: 8.0


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 493, 32)           5408      
                                                                 
 batch_normalization (BatchN  (None, 493, 32)          128       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 246, 32)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 239, 16)           4112      
                                                                 
 batch_normalization_1 (Batc  (None, 239, 16)          64        
 hNormalization)                                                 
                                                        